# 학습목표
  1. graphlab create를 사용한 추천

* Movie lens 평점 데이터 셋
 - http://grouplens.org/datasets/movielens/100k/

* load user, rating, items

In [1]:
import pandas as pd

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols,
 encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols,
 encoding='latin-1')

i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols,
 encoding='latin-1')

In [2]:
users

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
7,8,36,M,administrator,05201
8,9,29,M,student,01002
9,10,53,M,lawyer,90703


In [ ]:
items

In [ ]:
ratings

In [3]:
users.shape

(943, 5)

In [4]:
ratings.shape

(100000, 4)

In [5]:
items.shape

(1682, 24)

* load train / test data

In [6]:
cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings_train = pd.read_csv('ml-100k/ua.base', sep='\t', names = cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names = cols, encoding='latin-1')


In [7]:
print ratings_train.shape
print ratings_test.shape

(90570, 4)
(9430, 4)


* graphlab register & download
 - https://turi.com/download/academic.html
 - email, key 발급
 - e.g) https://turi.com/download/install-graphlab-create-command-line.html?email=macmath22%40gmail.com&key=3F03-279E-306F-D384-CBF7-1252-XXXX-XXXX

* train / test sframe 생성

In [11]:
import graphlab

In [15]:
train_data = graphlab.SFrame(ratings_train)
test_data = graphlab.SFrame(ratings_test)

AttributeError: 'module' object has no attribute 'SFrame'

In [ ]:
train_data

* popularity based recommendation

In [ ]:
popularity_model = graphlab.popularity_recommender.create(train_data, 
                                                        user_id='user_id', 
                                                        item_id='movie_id', 
                                                          target='rating')

In [ ]:
popularity_recomm = popularity_model.recommend(users = range(1, 6), k = 10)
popularity_recomm.print_rows(num_rows = 50)

연습문제)
 1. ratings_train에서 평균 평점이 가장 높은 영화 10개를 출력하세요

In [ ]:
ratings_train.groupby('movie_id')['rating'].mean().sort_values(ascending = False).head(10)

* Collaborative Filtering 사용
 - User-based, Item-based
 - 유사도를 이용하여 weighted-sum을 계산

* Graphlab에서 지원하는 유사도 
 1. Jaccard
 2. Cosine
 3. Pearson
 https://turi.com/products/create/docs/generated/graphlab.recommender.item_similarity_recommender.ItemSimilarityRecommender.html

In [ ]:
item_sim_model = graphlab.item_similarity_recommender.create(train_data, 
                                                        user_id='user_id', 
                                                        item_id='movie_id', 
                                                        target='rating', 
                                                similarity_type='cosine')

item_sim_recomm = item_sim_model.recommend(users=range(1,6), k = 10)

In [ ]:
item_sim_recomm.print_rows(50)

* evaluation
 - recall & precision
 - recall : 사용자가 좋아한다고 한 아이템을 몇개나 추천했는가?
 - precision : 추천된 아이템 중에서 몇개나 좋아하는가?
 
 - Recall만 고려할 경우, recall을 올리기 위해서는 모든 아이템을 추천하면 됨
 - 이 경우, precision이 현저하게 낮아짐
 - 높은 precision과 recall이 동시에 요구됨
 - 가장 이상적인 경우 precision = recal = 1

In [8]:
model_performance = graphlab.compare(test_data, [popularity_model, item_sim_model])
graphlab.show_comparison(model_performance,[popularity_model, item_sim_model])

NameError: name 'graphlab' is not defined

* graphlab classification & regression example
 - yelp review data

In [ ]:
import graphlab as gl
data =  gl.SFrame('https://static.turi.com/datasets/regression/yelp-data.csv')

In [ ]:
data

In [ ]:
data['is_good'] = data['stars'] >= 3

train_data, test_data = data.random_split(0.8)

model = gl.classifier.create(train_data, target='is_good',
                             features = ['user_avg_stars',
                                         'business_avg_stars',
                                         'user_review_count',
                                         'business_review_count'])

predictions = model.classify(test_data)

results = model.evaluate(test_data)


In [ ]:
graphlab.linear_regression

model = gl.regression.create(train_data, target='stars',
                                    features = ['user_avg_stars',
                                                'business_avg_stars',
                                                'user_review_count',
                                                'business_review_count'])

predictions = model.predict(test_data)
print predictions
results = model.evaluate(test_data)
print results